In [45]:
#https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews/data
#https://www.kaggle.com/code/stpeteishii/book-review-text-to-summary-bart-fit-predict

In [46]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
import nltk
#nltk.download('punkt_tab')
#nltk.download('stopwords')
print(os.getcwd())
import importlib
import re
import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.tensorboard import SummaryWriter
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report

c:\Users\oscar\SKOLA\D7047e\Team-19\Lab 1


In [47]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the Feedforward Neural Network
class SentimentClassifier(nn.Module):
    def __init__(self, input_size, hidden_size = 128, output_size = 6):
        super(SentimentClassifier, self).__init__()
        self.inlayer = nn.Linear(input_size, 2*hidden_size)   # First hidden layer
        self.hidden1 = nn.Linear(2*hidden_size, hidden_size)          # Output layer (0 to 5 stars)
        self.hidden2 = nn.Linear(hidden_size, hidden_size // 2)
        self.hidden3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.outlayer = nn.Linear(hidden_size // 4, output_size)          # Second hidden layer 
        self.droput5 = nn.Dropout()                                # Dropout layer (50%)
        self.droput3 = nn.Dropout(.3)
    def forward(self, x):
        x = F.relu(self.inlayer(x))
        x = self.droput5(x)
        x = F.relu(self.hidden1(x))
        x = self.droput3(x)
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.outlayer(x)

        return x


In [49]:
data=pd.read_csv('Books_rating.csv')
print(len(data))
display(data[0:1].T)
data=data[0:1000]

3000000


,0
Id,1882931173
Title,Its Only Art If Its Well Hung!
Price,NaN
User_id,AVCGYZL8FQQTD
profileName,"Jim of Oz ""jim-of-oz"""
review/helpfulness,7/7
review/score,4.0
review/time,940636800
review/summary,Nice collection of Julie Strain images
review/text,This is only for Julie Strain fans. It's a col...


In [50]:
inputt=data['review/text']
labelt=data['review/score']

print(inputt[0])

This is only for Julie Strain fans. It's a collection of her photos -- about 80 pages worth with a nice section of paintings by Olivia.If you're looking for heavy literary content, this isn't the place to find it -- there's only about 2 pages with text and everything else is photos.Bottom line: if you only want one book, the Six Foot One ... is probably a better choice, however, if you like Julie like I like Julie, you won't go wrong on this one either.


In [51]:


training_data, validation_data, training_labels, validation_labels = train_test_split(
    inputt, labelt, test_size=0.2, random_state=42
)

In [52]:
for i, text in enumerate(inputt):
    if not isinstance(text, str):  # Convert non-strings to empty string
        text = str(text) if text is not None else ''
    
    text = text.lower()
    text = re.sub(r'[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', text)  # Remove emails
    text = re.sub(r'((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', text)  # Remove IPs
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\d+(\.\d+)?', '', text) #Remove numbers
    inputt[i] = text  # Update the list with cleaned text

labelt = [round(x) for x in labelt]


print(labelt[0])
print(inputt[0])

C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

4
this is only for julie strain fans its a collection of her photos  about  pages worth with a nice section of paintings by oliviaif youre looking for heavy literary content this isnt the place to find it  theres only about  pages with text and everything else is photosbottom line if you only want one book the six foot one  is probably a better choice however if you like julie like i like julie you wont go wrong on this one either


C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputt[i] = text  # Update the list with cleaned text
C:\Users\oscar\AppData\Local\Temp\ipykernel_12744\1294964487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [53]:
# vectorize data using TFIDF and transform for PyTorch for scalability
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
training_data = training_data.todense()                             # convert to dense matrix for Pytorch
vocab_size = len(word_vectorizer.vocabulary_)
validation_data = word_vectorizer.transform(validation_data)
validation_data = validation_data.todense()
train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [54]:
# Create TensorDataset and DataLoader
batch_size = 128

train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = TensorDataset(validation_x_tensor, validation_y_tensor)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [55]:
# Define model, loss function, and optimizer
input_size = train_x_tensor.shape[1]  # TF-IDF vector size
print(input_size)
model = SentimentClassifier(input_size)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Optimizer

50000


In [56]:
num_epochs = 100  # Train for x amount of epochs
writer = SummaryWriter(log_dir='runs/Lab1_test_axel_extended')
best_val_loss = float('inf')
best_model_state = None
patience = 5
epochs_no_improvement = 0
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()  # Clear gradients
        outputs = model(batch_x)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()
    

    model.eval()  # Set to evaluation mode
    total_val_loss = 0 
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient tracking for efficiency
        for batch_x, batch_y in validation_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            predictions = torch.argmax(outputs, dim=1)  # Get predicted class
            correct += (predictions == batch_y).sum().item()
            total_val_loss += loss.item()
            total += batch_y.size(0)

    accuracy = correct / total
    writer.add_scalar('Loss/Train', total_loss, epoch)
    writer.add_scalar('Loss/Validation', total_val_loss, epoch)
    writer.add_scalar('Accuracy/Validation', accuracy, epoch)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Traning Loss: {total_loss/len(train_loader):.4f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {total_val_loss/len(validation_loader):.4f}")

    if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0  # reset counter if there was an improvement
    else:
        epochs_no_improve += 1  # increment counter if there was no improvement

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

Validation Accuracy: 0.0000
Epoch 1/100, Traning Loss: 1.8161
Epoch 1/100, Validation Loss: 1.8139
Validation Accuracy: 0.0000
Epoch 2/100, Traning Loss: 1.8127
Epoch 2/100, Validation Loss: 1.8102
Validation Accuracy: 0.0000
Epoch 3/100, Traning Loss: 1.8088
Epoch 3/100, Validation Loss: 1.8062
Validation Accuracy: 0.0000
Epoch 4/100, Traning Loss: 1.8047
Epoch 4/100, Validation Loss: 1.8018
Validation Accuracy: 0.0000
Epoch 5/100, Traning Loss: 1.8007
Epoch 5/100, Validation Loss: 1.7967
Validation Accuracy: 0.0000
Epoch 6/100, Traning Loss: 1.7944
Epoch 6/100, Validation Loss: 1.7911
Validation Accuracy: 0.0000
Epoch 7/100, Traning Loss: 1.7882
Epoch 7/100, Validation Loss: 1.7846
Validation Accuracy: 0.0000
Epoch 8/100, Traning Loss: 1.7821
Epoch 8/100, Validation Loss: 1.7769
Validation Accuracy: 0.0250
Epoch 9/100, Traning Loss: 1.7736
Epoch 9/100, Validation Loss: 1.7683
Validation Accuracy: 0.4250
Epoch 10/100, Traning Loss: 1.7649
Epoch 10/100, Validation Loss: 1.7592
Validati

In [57]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6007 (pid 16976), started 2 days, 2:40:47 ago. (Use '!kill 16976' to kill it.)